In [37]:
# import kagglehub

# path = kagglehub.dataset_download("adisongoh/it-service-ticket-classification-dataset")

# print("Path to dataset files:", path)

In [38]:
path = "C:/Users/Yakina/.cache/kagglehub/datasets/adisongoh/it-service-ticket-classification-dataset/versions/1/all_tickets_processed_improved_v3.csv"
# os.listdir(path)

In [39]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [40]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras import Sequential, Input, Model, layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, AveragePooling2D, TextVectorization
from tensorflow.keras.layers import Dense, Flatten, Softmax, Dropout, Embedding, SimpleRNN, LSTM, GRU, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, Rescaling, RandomWidth, RandomHeight, Lambda
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler

In [41]:
df = pd.read_csv(path)
df

,Document,Topic_group
0,connection with icon icon dear please setup ic...,Hardware
1,work experience user work experience user hi w...,Access
2,requesting for meeting requesting meeting hi p...,Hardware
3,reset passwords for external accounts re expir...,Access
4,mail verification warning hi has got attached ...,Miscellaneous
...,...,...
47832,git space for a project issues with adding use...,Access
47833,error sent july error hi guys can you help out...,Miscellaneous
47834,connection issues sent tuesday july connection...,Hardware
47835,error cube reports sent tuesday july error hel...,HR Support


In [42]:
df['Topic_group'].value_counts()

Topic_group
Hardware                 13617
HR Support               10915
Access                    7125
Miscellaneous             7060
Storage                   2777
Purchase                  2464
Internal Project          2119
Administrative rights     1760
Name: count, dtype: int64

In [43]:
df.isnull().any()

Document       False
Topic_group    False
dtype: bool

In [44]:
df[df.duplicated()]

,Document,Topic_group


In [45]:
label_encoder = LabelEncoder()
df['Topic_group'] = label_encoder.fit_transform(df['Topic_group']).astype('int32')
label_encoder.classes_

array(['Access', 'Administrative rights', 'HR Support', 'Hardware',
       'Internal Project', 'Miscellaneous', 'Purchase', 'Storage'],
      dtype=object)

In [46]:
df.head()

,Document,Topic_group
0,connection with icon icon dear please setup ic...,3
1,work experience user work experience user hi w...,0
2,requesting for meeting requesting meeting hi p...,3
3,reset passwords for external accounts re expir...,0
4,mail verification warning hi has got attached ...,5


In [47]:
train_val_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Topic_group'])
train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=42, stratify=train_val_df['Topic_group'])

In [48]:
train_ds = tf.data.Dataset.from_tensor_slices((train_df['Document'].values, train_df['Topic_group'].values))
val_ds = tf.data.Dataset.from_tensor_slices((val_df['Document'].values, val_df['Topic_group'].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['Document'].values, test_df['Topic_group'].values))

batch_size = 32
train_ds = train_ds.batch(batch_size)
val_ds = val_ds.batch(batch_size)
test_ds = test_ds.batch(batch_size)

print(type(train_ds), type(val_ds), type(test_ds))

<class 'tensorflow.python.data.ops.batch_op._BatchDataset'> <class 'tensorflow.python.data.ops.batch_op._BatchDataset'> <class 'tensorflow.python.data.ops.batch_op._BatchDataset'>


In [49]:
for inputs, targets in train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'new starter bogota complete colombia dear please help completed form tester senior technician joining well please complete going his his work location thank compliance officer colombia', shape=(), dtype=string)
targets[0]: tf.Tensor(2, shape=(), dtype=int32)


In [50]:
text_only_train_ds = train_ds.map(lambda x, y: x)

In [51]:
max_length = 500
max_tokens = 20000

text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode="tf_idf",
)

text_vectorization.adapt(text_only_train_ds)

int_train_ds_tfidf = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds_tfidf = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds_tfidf = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [54]:
vocabulary = text_vectorization.get_vocabulary()

In [55]:
encoded_sentence = text_vectorization("Hi, pls pm help me thank you.")
print(encoded_sentence)

tf.Tensor([8.058755  0.        1.2755619 ... 0.        0.        0.       ], shape=(10901,), dtype=float32)


In [56]:
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)